In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '1'

import logging
import numexpr as ne
import numpy as np
import torch
import datetime
from ddopai.envs.pricing.dynamic import DynamicPricingEnv
from ddopai.envs.pricing.dynamic_RL2 import RL2DynamicPricingEnv
from ddopai.envs.pricing.dynamic_lag import LagDynamicPricingEnv
from ddopai.envs.pricing.dynamic_inventory import DynamicPricingInvEnv
from ddopai.envs.actionprocessors import ClipAction, RoundAction
from ddopai.agents.obsprocessors import ConvertDictSpace

from ddopai.experiments.experiment_functions_online import run_experiment 
from ddopai.experiments.meta_experiment_functions import *
import requests
import yaml
import re
import pandas as pd
import wandb
from copy import deepcopy
import warnings
import gc
#from mushroom_rl import core 
from ddopai.experiments.meta_core import Core
import pickle

In [2]:
logging_level = logging.INFO
logging.basicConfig(level=logging_level)

ne.set_num_threads(1)
torch.backends.cudnn.enabled = False
torch.set_num_threads(1)

set_warnings(logging.INFO) # turn off warnings for any level higher or equal to the input level
LIBRARIES_TO_TRACK = ["ddopai", "mushroom_rl"]
PROJECT_NAME = "pricing_cMDP_test"

ENVCLASS = DynamicPricingEnv
RESULTS_DIR = "results"
def get_ENVCLASS(class_name):
    if class_name == "DynamicPricingEnv":
        return DynamicPricingEnv
    elif class_name == "DynamicPricingInvEnv":
        return DynamicPricingInvEnv
    elif class_name == "LagDynamicPricingEnv":
        return LagDynamicPricingEnv
    elif class_name == "RL2DynamicPricingEnv":
        return RL2DynamicPricingEnv
    else:
        raise ValueError(f"Unknown class name {class_name}")

# Experiment preparations
## Set-up WandB
### Init WandB

In [3]:
project_name = "pricing_cMDP"


### Track library versions and git hash of experiment

# Experiment parameters

In [4]:
config_train, config_agent, config_env, AgentClass, agent_name = prep_experiment(
        PROJECT_NAME,
        LIBRARIES_TO_TRACK,
        config_train_name="config_train.yaml",
        config_agent_name="config_agent.yaml",
        config_env_name="config_env.yaml",
    )

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: timlachner. Use `wandb login --relogin` to force relogin


INFO:root:ddopai: 0.0.7
INFO:root:mushroom_rl: 1.10.1
INFO:root:Git hash: 0240bca214a0886c93a097164834526abebcdb70
INFO:root:Configuration file 'config_train.yaml' successfully loaded.
INFO:root:Configuration file 'config_agent.yaml' successfully loaded.
INFO:root:Configuration file 'config_env.yaml' successfully loaded.


In [5]:
if config_env['lag_window_params'].get("lag_window") is not None:
        for env_kwargs in config_env["env_kwargs"]:
            env_kwargs["lag_window"] = config_env['lag_window_params']['lag_window']
            env_kwargs["env_class"] = "LagDynamicPricingEnv"


In [6]:
#artifact = wandb.use_artifact('raw_data:latest')
#path = artifact.download()
#raw_data = pickle.load(open(os.path.join(path, 'raw_data.pkl'), 'rb'))

In [7]:
raw_data, val_index_start, test_index_start = get_online_data(
            config_env,
            overwrite=False
        )

In [8]:
raw_data[0][0][0]

array([1.        , 0.17042525, 0.02231759, 0.37522929, 0.41223724,
       0.01571206, 0.48614612, 0.42916275, 0.19386668, 0.10603631])

## Environment parameters

* Get the environment parameters from the config file 
* Overwrite the ```lag_window```parameter with the parameter specified in the agent, if it is specified (since lag window is provided by the environment, but a tunable hyperparameter of the agent)

In [ ]:
round_action = RoundAction(unit_size=config_env["unit_size"])
clip_action = ClipAction(low =config_env["setup_kwargs"][])
postprocessors = [round_action]

#ENVCLASS = get_ENVCLASS(config_env["env_class"])
#environment = set_up_env_online(ENVCLASS, raw_data, val_index_start, test_index_start, config_env, postprocessors)
environments = prepare_env_online(get_ENVCLASS=get_ENVCLASS, raw_data=raw_data, val_index_start=0, test_index_start=0, config_env=config_env, postprocessors=postprocessors)

In [10]:
environments[0].inv

array([2000.])

In [11]:
environments[0].observation_space

Dict('features': Box([1.0000000e+00 1.4287769e-04 3.8245969e-04 9.3830319e-04 2.1704179e-04
 2.1403281e-03 8.0802059e-04 1.9769531e-03 2.2935087e-03 2.8973455e-03], [1.         0.49980804 0.49928105 0.49585605 0.49503562 0.49853876
 0.49922723 0.49936512 0.4979667  0.4993368 ], (10,), float32), 'inventory': Box(0.0, 1.0, (1,), float32), 'prev_action': Box(0.0, 20.0, (1,), float32), 'prev_done': Box(0.0, 1.0, (1,), float32), 'prev_reward': Box(-inf, inf, (1,), float32))

In [12]:
obs, _ = environments[0].get_observation()
obs

{'features': array([1.        , 0.17042525, 0.02231759, 0.37522929, 0.41223724,
        0.01571206, 0.48614612, 0.42916275, 0.19386668, 0.10603631]),
 'inventory': array([[1.]], dtype=float32),
 'prev_action': array([0.], dtype=float32),
 'prev_reward': array([0.], dtype=float32),
 'prev_done': array([1.], dtype=float32)}

In [13]:
conv = ConvertDictSpace(keep_time_dim=False)
X = conv(environments[0].get_observation()[0])
X

array([1.        , 0.17042525, 0.02231759, 0.37522929, 0.41223724,
       0.01571206, 0.48614612, 0.42916275, 0.19386668, 0.10603631,
       1.        , 0.        , 0.        , 1.        ])

## Agent Parameter

In [14]:
logging.info(f"Agent: {agent_name}")
if agent_name in ["SAC", "PPORNN", "RL2PPO"]:
    obsprocessors = [ConvertDictSpace(keep_time_dim=False, )]
else:
    obsprocessors = []
if AgentClass.train_mode == "env_interaction":
    if "link" in config_agent:
        glm_link, price_function = set_up_agent(AgentClass, environments[0], config_agent)
        config_agent["g"] = glm_link
        config_agent["price_function"] = price_function
        
        del config_agent["link"]
    if agent_name == "Clairvoyant":
        agent = AgentClass(
        alpha=environments[0].alpha,
        beta=environments[0].beta,
        environment_info=environments[0].mdp_info,
        **config_agent
        )
    else:
        agent = AgentClass(
        environment_info=environments[0].mdp_info,
        obsprocessors=obsprocessors,
        **config_agent
        )

else:
    raise ValueError("Invalid train_mode for online training")

INFO:root:Agent: RL2PPO
INFO:root:Actor (RL²) network:


Layer (type:depth-idx)                   Output Shape              Param #
RL2RNNActor                              [1, 1, 1]                 --
├─RL2RNN: 1-1                            [1, 1, 1]                 --
│    └─SpecificRNNWrapperHS: 2-1         [1, 1, 64]                --
│    │    └─GRU: 3-1                     [1, 1, 64]                15,360
│    └─Sequential: 2-2                   [1, 1]                    --
│    │    └─Linear: 3-2                  [1, 64]                   4,160
│    │    └─ReLU: 3-3                    [1, 64]                   --
│    │    └─Dropout: 3-4                 [1, 64]                   --
│    │    └─Linear: 3-5                  [1, 32]                   2,080
│    │    └─ReLU: 3-6                    [1, 32]                   --
│    │    └─Dropout: 3-7                 [1, 32]                   --
│    │    └─Linear: 3-8                  [1, 1]                    33
Total params: 21,633
Trainable params: 21,633
Non-trainable params: 0
Total

INFO:root:Critic (RL²) network:


Layer (type:depth-idx)                   Output Shape              Param #
RL2RNNValue                              [1, 1, 1]                 --
├─RL2RNN: 1-1                            [1, 1, 1]                 --
│    └─SpecificRNNWrapperHS: 2-1         [1, 1, 64]                --
│    │    └─GRU: 3-1                     [1, 1, 64]                15,360
│    └─Sequential: 2-2                   [1, 1]                    --
│    │    └─Linear: 3-2                  [1, 64]                   4,160
│    │    └─ReLU: 3-3                    [1, 64]                   --
│    │    └─Dropout: 3-4                 [1, 64]                   --
│    │    └─Linear: 3-5                  [1, 32]                   2,080
│    │    └─ReLU: 3-6                    [1, 32]                   --
│    │    └─Dropout: 3-7                 [1, 32]                   --
│    │    └─Linear: 3-8                  [1, 1]                    33
Total params: 21,633
Trainable params: 21,633
Non-trainable params: 0
Total

In [15]:
earlystoppinghandler = set_up_earlystoppinghandler(config_train)

In [16]:
environments[0].step(np.array([5]))

({'features': array([1.        , 0.00350445, 0.09968054, 0.08908962, 0.00783165,
         0.42725708, 0.24546228, 0.4031461 , 0.03825726, 0.02094546]),
  'inventory': array([[0.99793303]], dtype=float32),
  'prev_action': array([[5.]], dtype=float32),
  'prev_reward': array([[20.669678]], dtype=float32),
  'prev_done': array([0.], dtype=float32)},
 array([20.66967708]),
 array([False]),
 {'inv': array([1995.86606026]),
  'demand': array([4.13393542]),
  'demand_noise_free': array([4.96748428]),
  'action': array([5.]),
  'reward': array([20.66967708])})

In [17]:
dataset = run_experiment(
        agent,
        environments[:1],
        n_epochs=config_train["n_epochs"],
        n_steps=config_train["n_steps"],
        n_steps_per_fit=None,
        n_episodes_per_fit=1,
        early_stopping_handler=earlystoppinghandler,
        save_best=config_train["save_best"],
        run_id=wandb.run.id,
        tracking="wandb",
        eval_step_info=False,
        print_freq=1,
        results_dir = RESULTS_DIR,
        return_dataset=True,
        return_score=False
    )

INFO:root:Starting experiment


Experiment directory: results/5kdasfgd


  0%|          | 1/400 [00:00<00:48,  8.15it/s]

advs (204, 1)
vpreds (204, 1)
tdlam_rets (204, 1)


INFO:root:Epoch 2: R=[13.93980706], J=[13.93980706]
  0%|          | 2/400 [00:00<00:46,  8.56it/s]

...

advs (69, 1)
vpreds (69, 1)
tdlam_rets (69, 1)


INFO:root:Epoch 57: R=[-119.45594048], J=[-119.45594048]


advs (70, 1)
vpreds (70, 1)
tdlam_rets (70, 1)


 14%|█▍        | 57/400 [00:04<00:26, 13.10it/s]


KeyboardInterrupt: 

In [ ]:
wandb.finish()

Action,▄▇▁▆▄▃▅▄▄▂▅▆▆▄▃▇▄▇▆▇▆▆▃▄▄▁▆▅█▆▄▇▆▇██▂▄▆▆
Cumulative_Reward,▁▁▁▁▅▃▃▂▄▅▂▄▄▂▃▃▂▄▂▄▃▃▆▃▂▂▅▄▂▅▅▂▂▃▃▂▆▅▆█
Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
Inventory,▆█▂▃▂▅▇▂█▁▄▂▁▃▅▇▃██▄▆▅▇▆▁▃▂█▄▃▂█▆▄▃▁▅▁▂█
Reward,▆▇▄▆▄▄▅▆▅█▅▅█▆▅▅▆▆▆▆▆▁▆▄▇▅█▄█▆▅▅▆▃▁▆▂▃▆▅
t,▃▂▂▃▁▂▂▁▆█▆▃▇▇▂▁▄▇▁▃▂▇▆▃▇▄▅▆▇█▅▄▁▅▇▂▅▂▄▁
Action,0.71
Cumulative_Reward,1184.4423
Epoch,399
Inventory,0
Reward,3.26642
